In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

# --- 1. Define Constants ---
# File paths based on your directory structure
MIMIC_ICU_DIR = 'data/mimic-iv-clinical-database-demo-2.2/icu/'
MIMIC_HOSP_DIR = 'data/mimic-iv-clinical-database-demo-2.2/hosp/'

# Define the time window for bucketing data (in hours)
TIME_BUCKET_SIZE = 4

# --- Define itemids for the features you want ---
# States (Vitals & Labs)
ID_HEART_RATE = 220045
ID_SYSTOLIC_BP = 220179
ID_DIASTOLIC_BP = 220180
ID_TEMPERATURE = 223761
ID_LACTATE = 50813 # from d_labitems

# Actions (Medications)
ID_NOREPINEPHRINE = 221906 # Vasopressor
ID_FLUID_BOLUS = 225158     # IV Fluid

print("Cell 1: Setup and constants defined successfully.")

Cell 1: Setup and constants defined successfully.


In [2]:
# --- 2. Load and Merge Data Upfront ---
print("Loading and filtering raw data...")

# ICU Stays (the source of truth for stay_id and intime)
df_icu_stays = pd.read_csv(f'{MIMIC_ICU_DIR}icustays.csv.gz', usecols=['stay_id', 'subject_id', 'intime'])
df_icu_stays['intime'] = pd.to_datetime(df_icu_stays['intime'])

# Vitals from chartevents
df_chartevents = pd.read_csv(f'{MIMIC_ICU_DIR}chartevents.csv.gz', usecols=['stay_id', 'charttime', 'itemid', 'valuenum'])
state_vitals_ids = [ID_HEART_RATE, ID_SYSTOLIC_BP, ID_DIASTOLIC_BP, ID_TEMPERATURE]
df_vitals_raw = df_chartevents[df_chartevents['itemid'].isin(state_vitals_ids)].copy()

# Labs from labevents
df_labevents = pd.read_csv(f'{MIMIC_HOSP_DIR}labevents.csv.gz', usecols=['subject_id', 'charttime', 'itemid', 'valuenum'])
state_labs_ids = [ID_LACTATE]
df_labs_raw = df_labevents[df_labevents['itemid'].isin(state_labs_ids)].copy()

# Actions from inputevents
df_inputevents = pd.read_csv(f'{MIMIC_ICU_DIR}inputevents.csv.gz', usecols=['stay_id', 'starttime', 'itemid', 'amount'])
action_ids = [ID_NOREPINEPHRINE, ID_FLUID_BOLUS]
df_actions_raw = df_inputevents[df_inputevents['itemid'].isin(action_ids)].copy()

# --- Centralize all merges here ---
# Add 'intime' to vitals and actions using their 'stay_id'
df_vitals = pd.merge(df_vitals_raw, df_icu_stays, on='stay_id', how='inner')
df_actions = pd.merge(df_actions_raw, df_icu_stays, on='stay_id', how='inner')

# Add 'stay_id' and 'intime' to labs using their 'subject_id'
df_labs = pd.merge(df_labs_raw, df_icu_stays, on='subject_id', how='inner')

print("Cell 2: Raw data loaded and merged successfully.")

Loading and filtering raw data...
Cell 2: Raw data loaded and merged successfully.


In [3]:
# --- 3. Simplified Time-Bucket and Aggregate ---
def simplified_process_and_aggregate(df, time_col, value_col, item_id, feature_name):
    """A simplified function that assumes df already has stay_id and intime."""
    df_filtered = df[df['itemid'] == item_id].copy()
    df_filtered[time_col] = pd.to_datetime(df_filtered[time_col])
    
    # Calculate hours from admission (intime is already present)
    df_filtered['hours_in'] = (df_filtered[time_col] - df_filtered['intime']).dt.total_seconds() / 3600
    df_filtered['time_bucket'] = (df_filtered['hours_in'] // TIME_BUCKET_SIZE).astype(int)
    
    # Aggregate by taking the mean value in each bucket
    df_agg = df_filtered.groupby(['stay_id', 'time_bucket'])[value_col].mean().reset_index()
    df_agg = df_agg.rename(columns={value_col: feature_name})
    
    return df_agg

print("Aggregating data into time buckets...")
# Process States
df_hr = simplified_process_and_aggregate(df_vitals, 'charttime', 'valuenum', ID_HEART_RATE, 'heart_rate')
df_sbp = simplified_process_and_aggregate(df_vitals, 'charttime', 'valuenum', ID_SYSTOLIC_BP, 'systolic_bp')
df_dbp = simplified_process_and_aggregate(df_vitals, 'charttime', 'valuenum', ID_DIASTOLIC_BP, 'diastolic_bp')
df_temp = simplified_process_and_aggregate(df_vitals, 'charttime', 'valuenum', ID_TEMPERATURE, 'temperature')
df_lactate = simplified_process_and_aggregate(df_labs, 'charttime', 'valuenum', ID_LACTATE, 'lactate')

# Process Actions
df_norepi = simplified_process_and_aggregate(df_actions, 'starttime', 'amount', ID_NOREPINEPHRINE, 'norepinephrine')
df_fluids = simplified_process_and_aggregate(df_actions, 'starttime', 'amount', ID_FLUID_BOLUS, 'fluid_bolus')

print("Cell 3: Data aggregation complete.")

Aggregating data into time buckets...
Cell 3: Data aggregation complete.


In [4]:
# --- 4. Merge into a Master DataFrame ---
print("Merging features into a master DataFrame...")
dfs_to_merge = [df_hr, df_sbp, df_dbp, df_temp, df_lactate, df_norepi, df_fluids]

# Start with the first DataFrame in the list as our base
df_master = dfs_to_merge[0]

# Loop through the rest of the DataFrames and merge them one by one
for df_to_merge in dfs_to_merge[1:]:
    df_master = pd.merge(df_master, df_to_merge, on=['stay_id', 'time_bucket'], how='outer')

# Sort for chronological order
df_master = df_master.sort_values(by=['stay_id', 'time_bucket']).reset_index(drop=True)

print("Cell 4: Master DataFrame created.")
df_master.info()

Merging features into a master DataFrame...
Cell 4: Master DataFrame created.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3970 entries, 0 to 3969
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   stay_id         3970 non-null   int64  
 1   time_bucket     3970 non-null   int64  
 2   heart_rate      3122 non-null   float64
 3   systolic_bp     2095 non-null   float64
 4   diastolic_bp    2095 non-null   float64
 5   temperature     2725 non-null   float64
 6   lactate         1307 non-null   float64
 7   norepinephrine  376 non-null    float64
 8   fluid_bolus     1300 non-null   float64
dtypes: float64(7), int64(2)
memory usage: 279.3 KB


In [5]:
# --- FINAL CORRECTED Cell 5: Handle Missing Data ---
print("Handling missing data...")

# This .apply() method correctly forward-fills within each group...
# ...and .reset_index(drop=True) removes the ambiguous multi-index.
df_master = df_master.groupby('stay_id').apply(lambda group: group.ffill()).reset_index(drop=True)

# Fill any remaining NaNs (especially at the beginning of a stay) with 0
df_master = df_master.fillna(0)

print("Cell 5: Missing data handled, and index has been reset.")

Handling missing data...
Cell 5: Missing data handled, and index has been reset.


C:\Users\ivinm\AppData\Local\Temp\ipykernel_32952\3975144854.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_master = df_master.groupby('stay_id').apply(lambda group: group.ffill()).reset_index(drop=True)


In [6]:
# --- DEBUGGING CELL ---
# Insert this cell right before the original Cell 6 to inspect the DataFrame

print("--- Inspecting df_master just before the error ---")
print("\nFirst 5 rows:")
print(df_master.head())

print("\n\nDataFrame Info:")
df_master.info()

print(f"\n\nIs 'stay_id' a column? -> {'stay_id' in df_master.columns}")

--- Inspecting df_master just before the error ---

First 5 rows:
    stay_id  time_bucket  heart_rate  systolic_bp  diastolic_bp  temperature  \
0  30057454           -1         0.0          0.0           0.0         0.00   
1  30057454            0       105.8          0.0           0.0        98.25   
2  30057454            1       108.0          0.0           0.0        98.70   
3  30057454            2       109.5          0.0           0.0        97.70   
4  30057454            3       113.0          0.0           0.0        97.70   

   lactate  norepinephrine  fluid_bolus  
0      0.9             0.0     0.000000  
1      0.9             0.0    17.916667  
2      0.9             0.0    17.916667  
3      0.9             0.0     7.000000  
4      0.9             0.0    31.416668  


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3970 entries, 0 to 3969
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          -----------

In [7]:
# --- 6. Create Final S_t, A_t, S_{t+1} Table ---
print("Creating final S_t, A_t, S_{t+1} table...")
state_cols = ['heart_rate', 'systolic_bp', 'diastolic_bp', 'temperature', 'lactate']
action_cols = ['norepinephrine', 'fluid_bolus']

# Create the "next state" columns by shifting the state columns up by one
for col in state_cols:
    df_master[f'{col}_next'] = df_master.groupby('stay_id')[col].shift(-1)

# Drop the last row for each patient, as it has no "next state"
df_final = df_master.dropna().reset_index(drop=True)

print("Cell 6: Preprocessing complete!")
print(f"Created a final dataset with {len(df_final)} samples.")
df_final.head()

Creating final S_t, A_t, S_{t+1} table...
Cell 6: Preprocessing complete!
Created a final dataset with 3830 samples.


,stay_id,time_bucket,heart_rate,systolic_bp,diastolic_bp,temperature,lactate,norepinephrine,fluid_bolus,heart_rate_next,systolic_bp_next,diastolic_bp_next,temperature_next,lactate_next
0,30057454,-1,0.0,0.0,0.0,0.00,0.9,0.0,0.000000,105.80,0.0,0.0,98.25,0.9
1,30057454,0,105.8,0.0,0.0,98.25,0.9,0.0,17.916667,108.00,0.0,0.0,98.70,0.9
2,30057454,1,108.0,0.0,0.0,98.70,0.9,0.0,17.916667,109.50,0.0,0.0,97.70,0.9
3,30057454,2,109.5,0.0,0.0,97.70,0.9,0.0,7.000000,113.00,0.0,0.0,97.70,0.9
4,30057454,3,113.0,0.0,0.0,97.70,0.9,0.0,31.416668,110.75,82.5,56.0,97.70,0.9


In [8]:
# --- 7. Build and Train the World Model ---
# X includes the state and action at time 't'
X = df_final[state_cols + action_cols]
# Y is the state at time 't+1'
Y = df_final[[f'{col}_next' for col in state_cols]]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the XGBoost regressor
xgbr = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# Wrap it with MultiOutputRegressor to handle multiple targets
multi_output_model = MultiOutputRegressor(estimator=xgbr)

print("Training the world model...")
multi_output_model.fit(X_train, y_train)
print("Cell 7: Training complete.")

Training the world model...
Cell 7: Training complete.


In [9]:
# --- Save the trained world model (XGBoost MultiOutputRegressor) ---
import joblib

joblib.dump(multi_output_model, 'world_model.pkl')
print("World model saved as 'world_model.pkl'.")

World model saved as 'world_model.pkl'.


In [ ]:
# --- 8. Evaluate the World Model ---
print("Evaluating the world model...")
y_pred = multi_output_model.predict(X_test)

# Calculate overall Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"\nOverall World Model MSE on Test Set: {mse:.4f}")

# You can also look at the error for each variable individually
print("\n--- MSE for each predicted variable ---")
for i, col in enumerate(state_cols):
    var_mse = mean_squared_error(y_test.iloc[:, i], y_pred[:, i])
    print(f"  - MSE for {col}_next: {var_mse:.4f}")

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class PatientSimulatorEnv(gym.Env):
    """A custom Gym environment for simulating patient treatment."""
    
    def __init__(self, world_model, initial_states_df, state_cols, action_cols):
        super(PatientSimulatorEnv, self).__init__()
        
        self.world_model = world_model
        self.initial_states = initial_states_df[state_cols].values
        self.state_cols = state_cols
        self.action_cols = action_cols
        
        # Define action and observation space
        # They must be gym.spaces objects
        # Example: 5 discrete levels for norepinephrine, 5 for fluids = 25 actions
        self.action_space = spaces.Discrete(25) 
        
        # The state space is continuous
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, 
                                            shape=(len(state_cols),), dtype=np.float32)
        
        self.current_state = None
        self.episode_length = 0

    def reset(self, seed=None, options=None):
        # Randomly select an initial state from the real data
        initial_state_idx = np.random.randint(0, len(self.initial_states))
        self.current_state = self.initial_states[initial_state_idx]
        self.episode_length = 0
        
        return self.current_state, {} # Return state and an empty info dict

    def step(self, action):
        # 1. Decode the discrete action into medication dosages
        # Example: action #15 -> level 3 norepi, level 3 fluids
        # This requires you to define a mapping.
        # For now, let's assume a placeholder for the action vector.
        action_vector = np.zeros(len(self.action_cols)) # Placeholder
        
        # 2. Prepare the input for the world model
        model_input = np.concatenate([self.current_state, action_vector]).reshape(1, -1)
        
        # 3. Use the world model to predict the next state
        previous_lactate = self.current_state[self.state_cols.index('lactate')] # Get lactate before the step
        predicted_next_state = self.world_model.predict(model_input)[0]
        self.current_state = predicted_next_state
        self.episode_length += 1
        
        # 4. Calculate the reward and determine if the episode is done
        new_lactate = self.current_state[self.state_cols.index('lactate')]
        
        terminated = False
        
        # Reward is based on the change in lactate from the previous state
        reward = (previous_lactate - new_lactate) * 10 # Reward for reducing lactate

        
        # Add large penalties/rewards for terminal states
        if new_lactate > 4.0:
            reward -= 100 # Large penalty for critical state
            terminated = True
        elif new_lactate < 1.0 and self.episode_length > 5: # Give a bonus for stabilizing
             reward += 50
             terminated = True
            
        if self.episode_length >= 50:
            terminated = True
            
        return self.current_state, reward, terminated, False, {}

In [ ]:
from stable_baselines3 import DQN

# 1. Create an instance of your environment
# We use the 'X' DataFrame from the previous step to get initial states
env = PatientSimulatorEnv(multi_output_model, X, state_cols, action_cols)

# 2. Instantiate the DQN agent
model = DQN("MlpPolicy", env, verbose=1)

# 3. Train the agent
# This will take some time. The agent will play out thousands of episodes in the simulator.
print("Training RL agent...")
model.learn(total_timesteps=1000000, progress_bar=True)
print("Training complete.")

In [ ]:
# --- Evaluate the trained agent ---
episodes = 10
total_reward = 0

for ep in range(episodes):
    obs, info = env.reset()
    done = False
    ep_reward = 0
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        ep_reward += reward
    
    total_reward += ep_reward
    print(f"Episode {ep + 1}: Total Reward = {ep_reward}")

avg_reward = total_reward / episodes
print(f"\nAverage reward over {episodes} episodes: {avg_reward:.2f}")

In [ ]:
# --- Save the trained model ---
model.save("dqn_mimic_patient_model_v1")

print("Model saved successfully!")